In [44]:
import requests

In [31]:
BASE_URL = "http://77.239.108.6:6543"

In [36]:
# Получение chat_id (передаем непустой JSON, например {"dummy": true})
response = requests.get(f"{BASE_URL}/api/get_new_chat_id", json={"dummy": True})
data = response.json()
print("Полученный ответ:", data)
chat_id = data["chat_id"]
print("Получен chat_id:", chat_id)

Полученный ответ: {'chat_id': '0e8540a2-dcbe-4791-a3e9-bf405077de91', 'message': 'Здравствуйте. Меня зовут цифровой помощник, я представляю компанию ЭнергоСбыт. Звоню по вопросу задолженности по коммунальным услугам. В целях контроля качества разговор записывается. Подскажите, пожалуйста, я сейчас разговариваю со Степанчук Алена Сергеевна?'}
Получен chat_id: 0e8540a2-dcbe-4791-a3e9-bf405077de91


In [43]:
# # Отправка ответа "Да" для перехода в следующую фазу (идентификация -> debt_discussion)
response = requests.post(
    f"{BASE_URL}/api/chat",
    json={"chat_id": chat_id, "message": "А как можно оплатить через ваш сайт или мобильное приложение?"},
)
data = response.json()
print("Ответ бота после подтверждения:", data)


Ответ бота после подтверждения: {'chat_id': '0e8540a2-dcbe-4791-a3e9-bf405077de91', 'message': 'Оплатить коммунальные услуги можно через Личный кабинет на нашем сайте или мобильное приложение. Перейдите в раздел «Счета и оплаты», выберите способ оплаты (через СБП или банковской картой) и следуйте инструкциям на экране. Когда планируете оплатить?'}


In [ ]:
import requests

BASE_URL = "http://77.239.108.6:6543"

# 1. Создание нового чата с передачей ФИО и компании
response = requests.post(
    f"{BASE_URL}/api/get_new_chat_id",
    json={
        "full_name": "Каспаров Гарри Кимович",
        "company_name": "МосЭнерго"
    }
)
data = response.json()
print("Полученный ответ:", data)
chat_id = data["chat_id"]
print("Получен chat_id:", chat_id)

# 2. Отправка сообщения с динамическими переменными (все значения можно менять)
response = requests.post(
    f"{BASE_URL}/api/chat",
    json={
        "chat_id": chat_id,
        "message": "Здравствуйте, я Каспаров Гарри Кимович",
        "full_name": "Каспаров Гарри Кимович",
        "account_number": "555888333",
        "debt_amount": "99 999 руб.",
        "address": "Москва, ул. Победы, д. 1",
        "company_name": "МосЭнерго",
        "company_phone": "8-800-123-45-67",
        "partial_payment_amount": "50 000 руб."
    }
)
data = response.json()
print("Ответ бота:", data)


Полученный ответ: {'chat_id': 'd81bfdcb-118b-4312-8a1f-0a7d066a4428', 'message': 'Здравствуйте. Меня зовут цифровой помощник, я представляю компанию МосЭнерго. Звоню по вопросу задолженности по коммунальным услугам. В целях контроля качества разговор записывается. Подскажите, пожалуйста, я сейчас разговариваю c Каспаров Гарри Кимович?'}
Получен chat_id: d81bfdcb-118b-4312-8a1f-0a7d066a4428
Ответ бота: {'chat_id': 'd81bfdcb-118b-4312-8a1f-0a7d066a4428', 'message': 'На вашем лицевом счёте 555888333 по адресу Москва, ул. Победы, д. 1 образовалась задолженность в размере 99 999 руб. Скажите, пожалуйста, когда вы планируете произвести оплату?'}
